In [9]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table
from dash.dependencies import Input, Output
import base64


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pymongo import MongoClient


#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from animalShelterCRUD import AnimalShelter



###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name

username = "aacuser"
password = "password"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object and accept projection json input
#df = pd.DataFrame.from_records(dumps(shelter.read({})))
df = pd.DataFrame.from_records(shelter.read({}))

#########################
# Dashboard Layout / View
#########################
app = JupyterDash('Dash DataTable Only') #SimpleExample

# for image
image_filename = 'GraziosoSalvareLogo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())
# for image

app.layout = html.Div([
    html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()), style={'width' : '100px'}),
    #html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('Ian Hefflefinger SNHU CS-340 Dashboard'))),
    html.Hr(),
    html.Div(className="row",
            style={"display" : "flex"},
            children=[
                dcc.RadioItems(
                    id='my_radio',
                    options=[
                        {'label': 'Water Rescue', 'value': 'WTR'},
                        {'label': 'Mountain/Wilderness Rescue', 'value': 'MWR'},
                        {'label': 'Disaster/Individual Tracking', 'value': 'DIT'},
                        {'label': 'Reset', 'value': 'reset'}
                    ],
                    value='reset'
                ) 
            ]),
    dash_table.DataTable(
        id='datatable-interactivity', #datatable-id
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        #FIXME: Set up the features for your interactive data table to make it user-friendly for your client
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable=False,
        row_deletable=False,
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current= 0,
        page_size= 10,
    ),
    html.Br(),
    html.Hr(),
    html.Div(
        id='map-id',
        className='col s12 m6',
    ),   
])

#############################################
# Interaction Between Components / Controller
#############################################

@app.callback(Output('datatable-interactivity', 'data'), [Input('my_radio', 'value')])

def on_click(radio):
    if (radio == 'WTR'):
        df = pd.DataFrame.from_records(shelter.read({'animal_type' : 'Dog', 'breed' : {'$in' : ['Labrador Retriever Mix', 'Chesapeake Bay Retriever', 'Newfoundland']}, 'sex_upon_outcome' : 'Intact Female', 'age_upon_outcome_in_weeks' : {'$gte' : 26}, 'age_upon_outcome_in_weeks' : {'$lte' : 156}}))
    elif (radio == 'MWR'):
        df = pd.DataFrame.from_records(shelter.read({'animal_type' : 'Dog', 'breed' : {'$in' : ['German Shepherd', 'Alaskan Malamute', 'Old English Sheepdog', 'Siberian Husky', 'Rottweiler']}, 'sex_upon_outcome' : 'Intact Male', 'age_upon_outcome_in_weeks' : {'$gte' : 26}, 'age_upon_outcome_in_weeks' : {'$lte' : 156}}))
    elif (radio == 'DIT'):
        df = pd.DataFrame.from_records(shelter.read({'animal_type' : 'Dog', 'breed' : {'$in' : ['Doberman Pinscher', 'German Shepherd', 'Golden Retriever', 'Bloodhound', 'Rottweiler']}, 'sex_upon_outcome' : 'Intact Male', 'age_upon_outcome_in_weeks' : {'$gte' : 20}, 'age_upon_outcome_in_weeks' : {'$lte' : 300}}))
    else: # no need to check for 'reset' because there is only one button not yet checked - the 'reset' button
        df = pd.DataFrame.from_records(shelter.read({}))
    
    return df.to_dict('records')


@app.callback(
    Output('datatable-interactivity', 'style_data_conditional'),
    [Input('datatable-interactivity', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


@app.callback(
    Output('map-id', "children"),
    [Input('datatable-interactivity', "derived_viewport_data")]) # 'datatable-id'

def update_map(viewData):
    dff = pd.DataFrame.from_dict(viewData)
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width' : '1000px', 'height' : '500px'}, center=[30.75, -97.48], zoom=10, children=[
            dl.TileLayer(id='base-layer-id'),
            # Marker with tooltip and popup
            # below code is not DRY, but will readdress if time permits *write for loop to add to children array
            dl.Marker(position=[dff.iloc[0,13],dff.iloc[0,14]], children=[ # this block represents one marker (coordinates will need to be dynamic based on the dataframe)
                dl.Tooltip(dff.iloc[0,4]), # [0, 4]
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[0,9]) #[1, 9] - previous code was incorrect (arrays start at zero)
                ])
            ]),
            dl.Marker(position=[dff.iloc[1,13],dff.iloc[1,14]], children=[ # this block represents one marker (coordinates will need to be dynamic based on the dataframe)
                dl.Tooltip(dff.iloc[1,4]), # [0, 4]
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[1,9]) #[1, 9] - previous code was incorrect (arrays start at zero)
                ])
            ]),
            dl.Marker(position=[dff.iloc[2,13],dff.iloc[2,14]], children=[ # this block represents one marker (coordinates will need to be dynamic based on the dataframe)
                dl.Tooltip(dff.iloc[2,4]), # [0, 4]
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[2,9]) #[1, 9] - previous code was incorrect (arrays start at zero)
                ])
            ]),
            dl.Marker(position=[dff.iloc[3,13],dff.iloc[3,14]], children=[ # this block represents one marker (coordinates will need to be dynamic based on the dataframe)
                dl.Tooltip(dff.iloc[3,4]), # [0, 4]
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[3,9]) #[1, 9] - previous code was incorrect (arrays start at zero)
                ])
            ]),
            dl.Marker(position=[dff.iloc[4,13],dff.iloc[4,14]], children=[ # this block represents one marker (coordinates will need to be dynamic based on the dataframe)
                dl.Tooltip(dff.iloc[4,4]), # [0, 4]
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[4,9]) #[1, 9] - previous code was incorrect (arrays start at zero)
                ])
            ]),
            dl.Marker(position=[dff.iloc[5,13],dff.iloc[5,14]], children=[ # this block represents one marker (coordinates will need to be dynamic based on the dataframe)
                dl.Tooltip(dff.iloc[5,4]), # [0, 4]
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[5,9]) #[1, 9] - previous code was incorrect (arrays start at zero)
                ])
            ]),
            dl.Marker(position=[dff.iloc[6,13],dff.iloc[6,14]], children=[ # this block represents one marker (coordinates will need to be dynamic based on the dataframe)
                dl.Tooltip(dff.iloc[6,4]), # [0, 4]
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[6,9]) #[1, 9] - previous code was incorrect (arrays start at zero)
                ])
            ]),
            dl.Marker(position=[dff.iloc[7,13],dff.iloc[7,14]], children=[ # this block represents one marker (coordinates will need to be dynamic based on the dataframe)
                dl.Tooltip(dff.iloc[7,4]), # [0, 4]
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[7,9]) #[1, 9] - previous code was incorrect (arrays start at zero)
                ])
            ]),
            dl.Marker(position=[dff.iloc[8,13],dff.iloc[8,14]], children=[ # this block represents one marker (coordinates will need to be dynamic based on the dataframe)
                dl.Tooltip(dff.iloc[8,4]), # [0, 4]
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[8,9]) #[1, 9] - previous code was incorrect (arrays start at zero)
                ])
            ]),
            dl.Marker(position=[dff.iloc[9,13],dff.iloc[9,14]], children=[ # this block represents one marker (coordinates will need to be dynamic based on the dataframe)
                dl.Tooltip(dff.iloc[9,4]), # [0, 4]
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[9,9]) #[1, 9] - previous code was incorrect (arrays start at zero)
                ])
            ])
            #replace above code with something a little more WET, if time permits
        ])
    ]


    
    

app

######################################
# queries for getting correct animals: *add .pretty()  -  if searching db manually through shell
######################################

# Water Rescue:
# db.animals.find({'animal_type' : 'Dog', 'breed' : {'$in' : ['Labrador Retriever Mix', 'Chesapeake Bay Retriever', 'Newfoundland']}, 'sex_upon_outcome' : 'Intact Female', 'age_upon_outcome_in_weeks' : {'$gte' : 26}, 'age_upon_outcome_in_weeks' : {'$lte' : 156}}).pretty()

# Mountain/Wilderness Rescue:
# db.animals.find({'animal_type' : 'Dog', 'breed' : {'$in' : ['German Shepherd', 'Alaskan Malamute', 'Old English Sheepdog', 'Siberian Husky', 'Rottweiler']}, 'sex_upon_outcome' : 'Intact Male', 'age_upon_outcome_in_weeks' : {'$gte' : 26}, 'age_upon_outcome_in_weeks' : {'$lte' : 156}}).pretty()

# Disaster/Individual Tracking:
# db.animals.find({'animal_type' : 'Dog', 'breed' : {'$in' : ['Doberman Pinscher', 'German Shepherd', 'Golden Retriever', 'Bloodhound', 'Rottweiler']}, 'sex_upon_outcome' : 'Intact Male', 'age_upon_outcome_in_weeks' : {'$gte' : 20}, 'age_upon_outcome_in_weeks' : {'$lte' : 300}}).pretty()



